In [2]:
!pip install gdown
!pip install transformers
!pip install opendatasets
!pip install pydub
!pip install gdown
!pip install -U sentence-transformers
!pip install resampy
!pip install matplotlib
!pip install pandas
!pip install nltk
!pip install tabulate
!pip install librosa
!pip install torch
!pip install torchvision
!pip install opencv-python
!pip install datasets
!pip install torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.7 MB/s eta 0:00:0000:0100:01


In [3]:
import torch
import resampy
from transformers import AutoProcessor, Wav2Vec2Model
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from time import time
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import os
from pydub import AudioSegment
import opendatasets as od
import pandas as pd
import gc
import random
import pickle
import gdown
import json
from nltk import word_tokenize
import string
from zipfile import ZipFile
from torch.nn.functional import cosine_similarity
from tabulate import tabulate
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import librosa
import librosa.display
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms.functional as F
from PIL import Image
import cv2
from sentence_transformers import SentenceTransformer
from IPython.display import FileLink
from shutil import rmtree
from datasets import load_dataset
from IPython.display import Audio, display
import torchaudio

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
url = "https://drive.google.com/file/d/1RSB9tOrbf0PLGAAVYx2Z4odRnWeAp8rS/view?usp=sharing"
output = "clasp.pt"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1RSB9tOrbf0PLGAAVYx2Z4odRnWeAp8rS
To: /kaggle/working/clasp.pt
100%|██████████| 25.4M/25.4M [00:00<00:00, 154MB/s] 


'clasp.pt'

In [14]:
url = "https://drive.google.com/file/d/1hXe01KlK6JUA62A3rIFAl-xwuF0Q9A4T/view?usp=sharing"
output = "sample_data.zip"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1hXe01KlK6JUA62A3rIFAl-xwuF0Q9A4T
From (redirected): https://drive.google.com/uc?id=1hXe01KlK6JUA62A3rIFAl-xwuF0Q9A4T&confirm=t&uuid=314b61f8-ef7a-4304-b3df-60f3fb59c929
To: /kaggle/working/sample_data.zip
100%|██████████| 34.6M/34.6M [00:00<00:00, 50.0MB/s]


'sample_data.zip'

In [15]:
with ZipFile('sample_data.zip', 'r') as zip_ref:
    zip_ref.extractall('samples-brown')

In [17]:
metadata_path = 'samples-brown/sample_metadata.json'
metadata = json.load(open(metadata_path))
metadata

[{'id': 45254,
  'file_path': 'dataset/part9/audios/audio_45254.wav',
  'text': 'Morgan hesitated , thinking that if this was a trick , it was a good one.',
  'category': 'adventure'},
 {'id': 45255,
  'file_path': 'dataset/part9/audios/audio_45255.wav',
  'text': "He didn't think it was possible for this couple to be pretending.",
  'category': 'adventure'},
 {'id': 45256,
  'file_path': 'dataset/part9/audios/audio_45256.wav',
  'text': 'The boy licked his dry lips.',
  'category': 'adventure'},
 {'id': 45257,
  'file_path': 'dataset/part9/audios/audio_45257.wav',
  'text': "He asked , `` Could we have a drink ''??",
  'category': 'adventure'},
 {'id': 19424,
  'file_path': 'dataset/part4/audios/audio_19424.wav',
  'text': 'In business circles , usually conservative , this sort of atmosphere would hardly be found.',
  'category': 'belles_lettres'},
 {'id': 19425,
  'file_path': 'dataset/part4/audios/audio_19425.wav',
  'text': 'But in our case -- and neither my wife nor I have extreme

In [18]:
vision_model = models.efficientnet_b7(pretrained=True)
vision_model.cuda()
_ = vision_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-c5b4e57e.pth
100%|██████████| 255M/255M [00:01<00:00, 228MB/s] 


In [34]:
from transformers import AutoProcessor, HubertModel
import soundfile as sf

hubert_processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
hubert_model = HubertModel.from_pretrained("facebook/hubert-large-ls960-ft").to(device)

Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
def hubert_audio_files(audio_file_list):
    embeddings = []
    for file_path in tqdm(audio_file_list):
        # Use soundfile to read the audio file
        data, samplerate = sf.read(file_path)
        # Normalize the audio data
        data = data / np.max(np.abs(data))
        # Resample the audio to 16kHz
        data = librosa.resample(data, orig_sr=samplerate, target_sr=16000)
        # Convert the audio data to PyTorch tensor
        audio = torch.from_numpy(data)
        # Process the audio data with Hubert processor
        inputs = hubert_processor(audio, sampling_rate=16000, return_tensors="pt").to(device)
        # Get the embeddings from Hubert model
        with torch.no_grad():
            hidden_states = hubert_model(**inputs).last_hidden_state
            # avg_embedding = torch.mean(hidden_states.cpu(), dim=1)
            # embeddings.append(avg_embedding.detach().numpy())
            avg_embedding = torch.mean(hidden_states, dim=1)
            embeddings.append(avg_embedding)
    return embeddings

In [36]:
import glob
import time

# List all .wav files in the samples-brown folder
audio_file_list = glob.glob('samples-brown/*.wav')

# Measure the time it takes to get embeddings
start_time = time.time()
embeddings = hubert_audio_files(audio_file_list)
end_time = time.time()

print(f"Time taken to get embeddings: {end_time - start_time} seconds")
# print(embeddings)

100%|██████████| 60/60 [00:03<00:00, 16.20it/s]


Time taken to get embeddings: 3.7080743312835693 seconds
[tensor([[ 0.0905,  0.4490,  0.2590,  ..., -0.1822, -0.1710, -0.0982]],
       device='cuda:0'), tensor([[ 0.1193,  0.3821,  0.1775,  ..., -0.1429, -0.2465, -0.1159]],
       device='cuda:0'), tensor([[ 0.1381,  0.3855,  0.2234,  ..., -0.1220, -0.2374,  0.0386]],
       device='cuda:0'), tensor([[ 0.1273,  0.4190,  0.2688,  ..., -0.1935, -0.1511,  0.0305]],
       device='cuda:0'), tensor([[ 0.1165,  0.4030,  0.2201,  ..., -0.0851, -0.2520,  0.1198]],
       device='cuda:0'), tensor([[ 0.0715,  0.3547,  0.2711,  ..., -0.0502, -0.3020, -0.0237]],
       device='cuda:0'), tensor([[ 0.1156,  0.3867,  0.2429,  ..., -0.1749, -0.2486,  0.0530]],
       device='cuda:0'), tensor([[ 0.2927,  0.4541,  0.2890,  ..., -0.1260, -0.1981,  0.1378]],
       device='cuda:0'), tensor([[ 0.0771,  0.4343,  0.2171,  ..., -0.1285, -0.2518,  0.0061]],
       device='cuda:0'), tensor([[ 0.2201,  0.3617,  0.1875,  ..., -0.2532, -0.1169,  0.0561]],
       

In [22]:
len(embeddings)

60

In [23]:
def get_image_embedding(path):
    image = cv2.imread(path)
    image_tensor = torch.from_numpy(image)
    image_tensor = image_tensor.unsqueeze(0)
    image_tensor = image_tensor.permute(0, 3, 1, 2)
    image_tensor = image_tensor.to(device)
    image_tensor = image_tensor.float()
    with torch.no_grad():
        output = vision_model(image_tensor)
        embedding = output[0].cpu()
        return embedding

def get_spectogram_embeddings_audio_files(audio_file_list):
    embeddings = []
    for file_path in tqdm(audio_file_list):
        y, sr = librosa.load(file_path)
        spec = librosa.stft(y)
        spec_db = librosa.amplitude_to_db(abs(spec))
        
        plt.clf()
        librosa.display.specshow(spec_db, x_axis='time', y_axis='log')
        plt.xlabel('')
        plt.ylabel('')
        plt.tight_layout()

        saved_path = f'spec.png'
        plt.savefig(saved_path, bbox_inches='tight', pad_inches=0)
        image_embeddings = get_image_embedding(saved_path)
        embeddings.append(image_embeddings)
    return embeddings

In [37]:
def get_image_embedding_batch(image_tensors):
    image_tensors = image_tensors.to(device).float()
    with torch.no_grad():
        output = vision_model(image_tensors)
    return output

def get_spectogram_embeddings_audio_files(audio_file_list):
    embeddings = []
    image_tensors = []
    
    for file_path in tqdm(audio_file_list):
        y, sr = librosa.load(file_path)
        spec = librosa.stft(y)
        spec_db = librosa.amplitude_to_db(abs(spec))
        
        plt.clf()
        librosa.display.specshow(spec_db, x_axis='time', y_axis='log')
        plt.xlabel('')
        plt.ylabel('')
        plt.tight_layout()

        # Convert the plot to a numpy array
        plt_canvas = plt.gcf().canvas
        plt_canvas.draw()
        image = np.frombuffer(plt_canvas.tostring_rgb(), dtype=np.uint8)
        image = image.reshape(plt_canvas.get_width_height()[::-1] + (3,))
        
        # Convert the numpy array to a tensor
        image_tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0)
        image_tensors.append(image_tensor)
    
    # Stack all image tensors and get embeddings in batch
    image_tensors = torch.cat(image_tensors, dim=0)
    embeddings = get_image_embedding_batch(image_tensors)
    
    return embeddings

In [ ]:
embeddings2 = get_spectogram_embeddings_audio_files(audio_file_list)

In [44]:
type(embeddings2[0])

torch.Tensor

In [45]:
type(embeddings[0])

torch.Tensor

In [28]:
## Hubert ASR
from transformers import AutoProcessor, HubertForCTC
import soundfile as sf

hubert_processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
hubert_model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft").to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClass

In [29]:
def asr_hubert_audio_files(audio_file_list):
    transcriptions = []
#     audio_raws = []
    for file_path in tqdm(audio_file_list):
        # Use soundfile to read the audio file
        data, samplerate = sf.read(file_path)
        # Normalize the audio data
        data = data / np.max(np.abs(data))
        # Resample the audio to 16kHz
        data = librosa.resample(data, orig_sr=samplerate, target_sr=16000)
        # Convert the audio data to PyTorch tensor
        audio = torch.from_numpy(data)
        # Process the audio data with Hubert processor
        inputs = hubert_processor(audio, sampling_rate=16000, return_tensors="pt").to(device)
        # Get the embeddings from Hubert model
        with torch.no_grad():
            logits = hubert_model(**inputs).logits
            predicted_ids = torch.argmax(logits, dim=-1)

            transcription = hubert_processor.batch_decode(predicted_ids)[0]
            transcriptions.append(transcription.lower())
#             audio_raws.append(audio)
    return transcriptions

In [30]:
start_time = time.time()
transcriptions = asr_hubert_audio_files(audio_file_list)
end_time = time.time()

print(f"Time taken to get embeddings: {end_time - start_time} seconds")

100%|██████████| 60/60 [00:04<00:00, 14.51it/s]

Time taken to get embeddings: 4.136927366256714 seconds


In [ ]:
text_model = SentenceTransformer('sentence-transformers/LaBSE').to(device)

In [40]:
transcriptions[0]

'something had to give'

In [42]:
text_embeddings = torch.Tensor(text_model.encode(transcriptions))

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [43]:
len(text_embeddings)

60

In [46]:
class HubertLabseConcat(nn.Module):
    def __init__(self, in_features_text, in_features_image):
        super(HubertLabseConcat, self).__init__()
        self.image_seq = nn.Sequential(
            nn.Linear(in_features_image, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(p=0.15),
            nn.Linear(768, 576),
            nn.BatchNorm1d(576),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(576, 768),
        )
        self.audio_seq = nn.Sequential(
            nn.Linear(in_features_text, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(p=0.15),
            nn.Linear(768, 576),
            nn.BatchNorm1d(576),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(576, 768),
        )
        
        self.mix_seq = nn.Sequential(
            nn.Linear(2 * 768, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 800),
            nn.LeakyReLU(),
            nn.Linear(800, 768),
        )
    
    def forward(self, x_audio, x_image):
        x1 = self.audio_seq(x_audio)
        x2 = self.image_seq(x_image)
        concats = torch.cat((x1, x2), dim=1)
        x = self.mix_seq(concats)
        return x

In [50]:
model_to_test = torch.load(f'clasp.pt', map_location=device)
model_to_test = model_to_test.to(device)

<ipython-input-50-dd0c054c599d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_to_test = torch.load(f'clasp.pt', map_location=device)


In [48]:
embeddings = torch.stack(embeddings)

In [ ]:
embeddings2 = torch.stack(embeddings2)

In [57]:
# Do not forget to normalize your spectogram and SSL embeddings before pass to fusion encoder for real inference.

start_time = time.time()
with torch.no_grad():
    final_emb = model_to_test(embeddings, embeddings2)
end_time = time.time()

print(f"Time taken to get embeddings: {end_time - start_time} seconds")

Time taken to get embeddings: 0.027492523193359375 seconds


In [ ]:
# preform simple inference on the model using cosimilarity

# Use the first text embedding as the sample
sample_embedding = text_embeddings[0].unsqueeze(0).to(device)

# Calculate cosine similarity between sample and all embeddings
cos_sim = nn.CosineSimilarity(dim=1)
similarities = cos_sim(sample_embedding, final_emb)

# Find the index of most similar embedding
most_similar_idx = torch.argmax(similarities).item()

print(f"Most similar embedding index: {most_similar_idx}")
print(f"Cosine similarity: {similarities[most_similar_idx].item():.4f}")


